In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
original_dataset_dir = '/Users/code/alzheimerDetect/clock_data_augmanted'
base_dir = '/Users/code/alzheimerDetect/split_clock_dataset'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Create directories for training and testing data
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Define categories
categories = ['0_no_clock', '1_severe_vis', '2_mod_vis_xhands', '3_hands_vis_errors','4_minor_VIS_errors','5_perfect_clock']

for category in categories:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(test_dir, category), exist_ok=True)

    # List only files in the category directory, ignore directories
    category_dir = os.path.join(original_dataset_dir, category)
    files = [f for f in os.listdir(category_dir) if os.path.isfile(os.path.join(category_dir, f))]
    
    # Split the data
    train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)
    
    # Copy files to train directory
    for file in train_files:
        src = os.path.join(category_dir, file)
        dst = os.path.join(train_dir, category, file)
        shutil.copyfile(src, dst)
    
    # Copy files to test directory
    for file in test_files:
        src = os.path.join(category_dir, file)
        dst = os.path.join(test_dir, category, file)
        shutil.copyfile(src, dst)

print("Dataset split into training and testing sets.")

Dataset split into training and testing sets.


In [3]:
# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2  # 20% of the data will be used for validation
)

test_datagen = ImageDataGenerator(
    rescale=1/255.0
)

In [4]:
# Load training data and validation data from the training directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to 150x150 pixels
    batch_size=32,
    class_mode='categorical',  # Adjusted for multi-class classification
    subset='training'  # Set as training data
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',  # Adjusted for multi-class classification
    subset='validation'  # Set as validation data
)

# Load test data from the test directory
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'  # Adjusted for multi-class classification
)

Found 3103 images belonging to 6 classes.
Found 773 images belonging to 6 classes.
Found 981 images belonging to 6 classes.


In [5]:
# Build the CNN Model
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # output multi-class classification
])


/opt/anaconda3/envs/tf_env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-06 12:42:12.440689: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-01-06 12:42:12.440743: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-01-06 12:42:12.440761: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-01-06 12:42:12.441072: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-06 12:42:12.441132: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:

In [6]:
# Compile the Model
model.compile(
    loss='categorical_crossentropy',  # Adjusted for multi-class classification
    optimizer='adam',
    metrics=['accuracy']
)

In [7]:
# Create an EarlyStopping callback
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

In [8]:
# Train the Model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[callback]
)

Epoch 1/20


2025-01-06 12:42:26.473540: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/opt/anaconda3/envs/tf_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


97/97 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - accuracy: 0.2100 - loss: 1.9028 - val_accuracy: 0.2549 - val_loss: 1.6675
Epoch 2/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.2761 - loss: 1.6287 - val_accuracy: 0.2768 - val_loss: 1.6315
Epoch 3/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.3670 - loss: 1.5125 - val_accuracy: 0.3014 - val_loss: 1.5980
Epoch 4/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.4774 - loss: 1.2875 - val_accuracy: 0.3312 - val_loss: 1.4779
Epoch 5/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.6010 - loss: 1.0359 - val_accuracy: 0.3648 - val_loss: 1.5439
Epoch 6/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.7140 - loss: 0.7589 - val_accuracy: 0.3855 - val_loss: 1.6129
Epoch 7/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.8022 - loss: 0.5528 - val_accuracy: 0.3661 - val_loss: 1.7454
Epoch 8/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.8715 - loss: 0.3736 - val_accuracy: 0.3441 - val_loss: 2.

In [10]:
# Evaluate the Model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.4f}')

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.4763 - loss: 1.2912
Test accuracy: 0.4669


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18496)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     4,735,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,281,076 (54.48 MB)

 Trainable params: 4,760,358 (18.16 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 9,520,718 (36.32 MB)

In [15]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Load the base model with ImageNet weights
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the base model layers to retain pre-trained weights
base_model.trainable = False

# Add custom layers for your classification task
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Reduces dimensionality while retaining information
    layers.Dropout(0.5),  # Helps reduce overfitting
    layers.Dense(128, activation='relu'),  # Add dense layer
    layers.Dropout(0.3),
    layers.Dense(6, activation='softmax')  # Output layer for 6 classes
])

In [16]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    callbacks=[early_stopping]
)

Epoch 1/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 19s 140ms/step - accuracy: 0.1980 - loss: 1.8067 - val_accuracy: 0.2044 - val_loss: 1.7608
Epoch 2/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.2046 - loss: 1.7715 - val_accuracy: 0.2044 - val_loss: 1.7567
Epoch 3/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.1811 - loss: 1.7794 - val_accuracy: 0.2044 - val_loss: 1.7581
Epoch 4/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.1831 - loss: 1.7741 - val_accuracy: 0.2044 - val_loss: 1.7559
Epoch 5/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 10s 99ms/step - accuracy: 0.2222 - loss: 1.7673 - val_accuracy: 0.2044 - val_loss: 1.7571
Epoch 6/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 10s 102ms/step - accuracy: 0.2109 - loss: 1.7642 - val_accuracy: 0.2044 - val_loss: 1.7562
Epoch 7/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.1920 - loss: 1.7630 - val_accuracy: 0.2044 - val_loss: 1.7565
Epoch 8/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.1951 - loss: 1.7678 - val_accur

In [18]:
# Evaluate the Model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.4f}')

31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step - accuracy: 0.1898 - loss: 1.7618
Test accuracy: 0.2029


In [19]:
# Unfreeze the top layers of the base model for fine-tuning
base_model.trainable = True

# Choose how many layers to fine-tune. For example, unfreeze the top 20 layers.
fine_tune_at = len(base_model.layers) - 20

# Freeze all layers up to the fine-tune point
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [20]:
# Recompile with a smaller learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
# Fine-tune the model
fine_tuning_history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,  # Additional epochs for fine-tuning
    callbacks=[early_stopping]
)

Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 24s 177ms/step - accuracy: 0.1613 - loss: 1.9213 - val_accuracy: 0.2044 - val_loss: 1.7586
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 14s 136ms/step - accuracy: 0.1862 - loss: 1.8124 - val_accuracy: 0.2044 - val_loss: 1.7589
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 13s 136ms/step - accuracy: 0.1946 - loss: 1.7948 - val_accuracy: 0.2044 - val_loss: 1.7569
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 14s 136ms/step - accuracy: 0.2018 - loss: 1.7820 - val_accuracy: 0.2044 - val_loss: 1.7561
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 14s 137ms/step - accuracy: 0.1943 - loss: 1.7793 - val_accuracy: 0.2044 - val_loss: 1.7560


In [ ]:
# Evaluate the Model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.4f}')